In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [1]:
import os
import re
import copy
import json
import time
import pickle
from multiprocessing import Pool
from datetime import datetime
from collections import defaultdict
from abc import ABC, abstractmethod


import torch
import gpytorch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.stats import norm
from rdkit import Chem
from botorch.models.gpytorch import GPyTorchModel
from botorch.fit import fit_gpytorch_model
from map4 import MAP4Calculator

from ga import SequenceGA, ScaffoldGA, GA
from baye import get_fitted_model, map4_fingerprint, TanimotoSimilarityKernel
from baye import AcqScoring, expected_improvement, probability_of_improvement
from helm import build_helm_string, parse_helm
from helm_genetic_operators import HELMGeneticOperators
from mhc import read_pssm_file, MHCIPeptideScorer

## Read dataset

In [3]:
mhci = pd.read_csv('../mhc/binding_data_2013/bdata.20130222.mhci.csv')
print(mhci[mhci['mhc_allele'].str.contains("HLA")]['mhc_allele'].unique().shape)

(119,)


In [4]:
# We removed those binding affinity values
# A lot of peptides were set with those values. Looks like some default values assigned...
dirty_values = [1, 2, 3, 5000, 10000, 20000, 43424, 50000, 69444.44444, 78125]

# Split dataset in training and testing sets
mhci = mhci[(mhci['mhc_allele'] == 'HLA-A*02:01') &
            (8 <= mhci['length']) &
            (mhci['length'] <= 11) &
            (~mhci['affinity_binding'].isin(dirty_values))]

## Genetic operators on HELM strings

In [5]:
with open('HELMCoreLibrary.json') as f:
    monomer_lib = json.load(f)

monomer_peptide_lib = [x for x in monomer_lib if x['polymerType'] == 'PEPTIDE']
AA1 = ["A", "R", "N", "D", "C", "E", "Q", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]
monomer_aa1_lib = [x for x in monomer_peptide_lib if x['symbol'] in AA1]

smiles = [monomer['smiles'] for monomer in monomer_aa1_lib]
fps = map4_fingerprint(smiles,input_type='smiles', radius=2)
t = TanimotoSimilarityKernel()
similarity_matrix = t.forward(fps, fps).numpy()

probability_matrix = []

for aa in similarity_matrix:
    tmp = aa.copy()
    tmp[tmp == 1.0] = 0
    probability_matrix.append(tmp / np.sum(tmp))
    
probability_matrix = np.array(probability_matrix)

helmgo = HELMGeneticOperators(monomer_aa1_lib, probability_matrix)

In [6]:
def affinity_binding_to_energy(value, input_unit='nM', temperature=300.):
    unit_converter = {'nM': 1e-9, 'uM': 1e-6, 'mM': 1e-3, 'M': 1}
    RT = 0.001987 * temperature
    return RT * np.log(value * unit_converter[input_unit])

def energy_to_affinity_binding(value, output_unit='nM', temperature=300.):
    unit_converter = {'nM': 1e9, 'uM': 1e6, 'mM': 1e3, 'M': 1}
    RT = 0.001987 * temperature
    return np.exp(value / RT) * unit_converter[output_unit]

## Generate random peptides

In [7]:
random_peptides = []
random_peptide_scores = []

n_peptides = [150]
peptide_length = [9]
energy_bounds = [-8.235, -4.944] # about between 1 uM and 250 uM
#energy_bounds = [-4.944, -4.531] # about between 250 uM and 500 uM
#energy_bounds = [-4.531, -4.118] # about between 500 uM and 1 mM
#energy_bounds = [-8.649, -8.235] # about between 500 nM and 1uM
energy_cutoff = -4.11 # 1 mM
#energy_cutoff = -4.944 # 250 uM
#energy_cutoff = -8.235 # 1 uM

pssm_files = ['../mhc/IEDB_MHC_I-2.9_matx_smm_smmpmbec/smmpmbec_matrix/HLA-A-02:01-8.txt',
              '../mhc/IEDB_MHC_I-2.9_matx_smm_smmpmbec/smmpmbec_matrix/HLA-A-02:01-9.txt',
              '../mhc/IEDB_MHC_I-2.9_matx_smm_smmpmbec/smmpmbec_matrix/HLA-A-02:01-10.txt',
              '../mhc/IEDB_MHC_I-2.9_matx_smm_smmpmbec/smmpmbec_matrix/HLA-A-02:01-11.txt']
mps = MHCIPeptideScorer(pssm_files, mhci, energy_cutoff=energy_cutoff)

# Generate random peptides
for n, size in zip(n_peptides, peptide_length):
    tmp_peptides, tmp_peptide_scores = mps.generate_random_peptides(n, energy_bounds, size)
    random_peptides.append(tmp_peptides)
    random_peptide_scores.append(tmp_peptide_scores)

random_peptides = np.concatenate(random_peptides)
random_peptide_scores = np.concatenate(random_peptide_scores)

clusters = defaultdict(list)
for i, sequence in enumerate(random_peptides):
    clusters[sequence.count('.')].append(i)
print('Distribution:', ['%d: %d' % (k, len(clusters[k])) for k in sorted(clusters.keys())])
print('')

print(len(random_peptides))
print(random_peptides)
print(random_peptide_scores)

----- Peptide global -----
N peptide: 8471
R2: 0.620
RMSD : 1.177 kcal/mol

Distribution: ['9: 150']

150
['PEPTIDE1{G.W.Y.A.W.W.Y.T.T}$$$$V2.0'
 'PEPTIDE1{M.L.S.N.Q.N.W.Q.D}$$$$V2.0'
 'PEPTIDE1{G.L.H.F.T.M.G.N.A}$$$$V2.0'
 'PEPTIDE1{L.I.P.V.W.Q.Y.L.S}$$$$V2.0'
 'PEPTIDE1{R.Y.V.E.S.T.L.H.L}$$$$V2.0'
 'PEPTIDE1{R.S.C.P.E.S.I.C.V}$$$$V2.0'
 'PEPTIDE1{K.M.F.V.F.Q.S.K.Y}$$$$V2.0'
 'PEPTIDE1{G.S.V.G.C.Y.F.N.A}$$$$V2.0'
 'PEPTIDE1{E.I.S.H.T.E.M.S.V}$$$$V2.0'
 'PEPTIDE1{N.F.I.H.D.C.N.I.V}$$$$V2.0'
 'PEPTIDE1{E.W.W.A.C.H.S.Y.L}$$$$V2.0'
 'PEPTIDE1{F.Q.T.K.I.C.M.T.G}$$$$V2.0'
 'PEPTIDE1{C.V.W.Q.D.G.F.T.F}$$$$V2.0'
 'PEPTIDE1{H.G.L.D.Q.W.V.L.I}$$$$V2.0'
 'PEPTIDE1{F.V.Y.H.G.W.K.L.W}$$$$V2.0'
 'PEPTIDE1{A.L.A.G.P.M.C.D.W}$$$$V2.0'
 'PEPTIDE1{P.S.F.Q.L.C.L.L.V}$$$$V2.0'
 'PEPTIDE1{M.Y.G.T.M.N.I.I.I}$$$$V2.0'
 'PEPTIDE1{M.H.C.W.I.Q.W.Y.T}$$$$V2.0'
 'PEPTIDE1{S.Y.T.A.F.D.W.R.A}$$$$V2.0'
 'PEPTIDE1{N.C.Y.L.E.L.H.I.V}$$$$V2.0'
 'PEPTIDE1{H.W.F.D.G.Y.H.Y.S}$$$$V2.0'
 'PEPTIDE1{M.V.Y.K.Y.A.S.D.C}$$$$V2.

In [124]:
random_peptide = 'PEPTIDE1{A.R.C.A.A.K.T.C.D}$PEPTIDE1,PEPTIDE1,8:R3-3:R3$$$V2.0'
c = helmgo.insert(random_peptide, only_terminus=False, maximum_size=12)

for i in c:
    print(i, i.count('.'))

PEPTIDE1{A.R.C.A.A.P.K.T.C.D}$PEPTIDE1,PEPTIDE1,9:R3-3:R3$$$V2.0 10
PEPTIDE1{A.R.Y.C.A.G.A.K.T.C.S.D}$PEPTIDE1,PEPTIDE1,10:R3-4:R3$$$V2.0 12
PEPTIDE1{R.A.R.R.C.A.A.K.T.C.D}$PEPTIDE1,PEPTIDE1,10:R3-5:R3$$$V2.0 11
PEPTIDE1{A.W.L.R.C.I.A.A.K.T.C.D}$PEPTIDE1,PEPTIDE1,11:R3-5:R3$$$V2.0 12
PEPTIDE1{A.R.C.A.A.K.R.T.C.D}$PEPTIDE1,PEPTIDE1,9:R3-3:R3$$$V2.0 10
PEPTIDE1{A.R.C.A.A.K.R.A.T.G.C.D}$PEPTIDE1,PEPTIDE1,11:R3-3:R3$$$V2.0 12
PEPTIDE1{A.R.C.A.A.Y.K.T.C.N.D}$PEPTIDE1,PEPTIDE1,9:R3-3:R3$$$V2.0 11
PEPTIDE1{A.H.R.C.A.P.A.K.T.C.D}$PEPTIDE1,PEPTIDE1,10:R3-4:R3$$$V2.0 11
PEPTIDE1{A.R.C.R.A.A.K.T.C.D}$PEPTIDE1,PEPTIDE1,9:R3-3:R3$$$V2.0 10
PEPTIDE1{A.R.C.R.A.A.Y.K.T.S.C.D}$PEPTIDE1,PEPTIDE1,11:R3-3:R3$$$V2.0 12


In [151]:
random_peptide = 'PEPTIDE1{A.R.C.A.A.K.T.C.D}$PEPTIDE1,PEPTIDE1,8:R3-3:R3$$$V2.0'
c = helmgo.delete(random_peptide, only_terminus=True, minimum_size=6, keep_connections=False)

for i in c:
    print(i, i.count('.'))

PEPTIDE1{A.R.C.A.A.K.T.C}$PEPTIDE1,PEPTIDE1,8:R3-3:R3$$$V2.0 8
PEPTIDE1{R.C.A.A.K.T.C.D}$PEPTIDE1,PEPTIDE1,7:R3-2:R3$$$V2.0 8
PEPTIDE1{A.R.C.A.A.K.T.C}$PEPTIDE1,PEPTIDE1,8:R3-3:R3$$$V2.0 8
PEPTIDE1{R.C.A.A.K.T.C}$PEPTIDE1,PEPTIDE1,7:R3-2:R3$$$V2.0 7
PEPTIDE1{R.C.A.A.K.T.C.D}$PEPTIDE1,PEPTIDE1,7:R3-2:R3$$$V2.0 8
PEPTIDE1{R.C.A.A.K.T.C.D}$PEPTIDE1,PEPTIDE1,7:R3-2:R3$$$V2.0 8
PEPTIDE1{R.C.A.A.K.T.C.D}$PEPTIDE1,PEPTIDE1,7:R3-2:R3$$$V2.0 8
PEPTIDE1{A.R.C.A.A.K.T.C}$PEPTIDE1,PEPTIDE1,8:R3-3:R3$$$V2.0 8
PEPTIDE1{A.R.C.A.A.K.T.C}$PEPTIDE1,PEPTIDE1,8:R3-3:R3$$$V2.0 8
PEPTIDE1{R.C.A.A.K.T.C}$PEPTIDE1,PEPTIDE1,7:R3-2:R3$$$V2.0 7


In [172]:
def boltzmann_probability(values, temperature=300.):
    p = np.exp(-(np.max(values) - values) / temperature)
    return p / np.sum(p)


values = np.linspace(0, 10, 11)
print(values)
v = np.floor(100 * boltzmann_probability(values, 0.1)).astype(int)
print(np.sum(v), v)

values = np.linspace(-10, 0, 11)
print(values)
v = np.floor(100 * boltzmann_probability(-values, 2)).astype(int)
print(np.sum(v), v)

values = np.ones(10) * 0.001
print(values)
v = np.floor(100 * boltzmann_probability(values, 2)).astype(int)
print(np.sum(v), v)

values = np.array([1])
print(values)
v = np.floor(100 * boltzmann_probability(values, 2)).astype(int)
print(np.sum(v), v)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
99 [ 0  0  0  0  0  0  0  0  0  0 99]
[-10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1.   0.]
94 [39 23 14  8  5  3  1  1  0  0  0]
[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001]
100 [10 10 10 10 10 10 10 10 10 10]
[1]
100 [100]


In [178]:
def _boltzmann_probability(values, temperature=300.):
    p = np.exp(-(np.max(values) - values) / temperature)
    return p / np.sum(p)


def _number_of_children_to_generate_per_parent(parent_scores, n_children, temperature, greater_is_better=True):
    """Compute the number of children generated by each parent sequence 
    based on their acquisition score using Boltzmann weighting.
    """
    i = 0

    if greater_is_better == False:
        parent_scores = -parent_scores

    # Boltzmann weighting
    children_per_parent = np.floor(n_children * _boltzmann_probability(parent_scores, temperature)).astype(int)
    
    # We are going to add 1 until the number of children is equal to n_children
    # but only when the number of children is higher than zero
    parent_indices = np.argwhere(children_per_parent > 0).flatten()
    
    while np.sum(children_per_parent) < n_children:
        try:
            if children_per_parent[parent_indices[i]] > 0:
                children_per_parent[parent_indices[i]] += 1
            i += 1
        except:
            # we restart from the beginning if we reach the end
            i = 0

    return children_per_parent

In [183]:
_number_of_children_to_generate_per_parent(np.linspace(-10, 0, 11), 250, 2, greater_is_better=False)

[10.  9.  8.  7.  6.  5.  4.  3.  2.  1. -0.]
[98 59 36 22 13  8  4  2  1  1  0]


array([99, 60, 37, 23, 14,  9,  4,  2,  1,  1,  0])

In [8]:
class DMTExperience:
    
    def __init__(self, init_population, init_scores):
        self._init_pop = init_population
        self._init_scores = init_scores
        
    def run(self, n_step=3, n_sample=10, **config):        
        data = []
        NCAN = config['NCAN']

        # Defined local and global GA optimization
        helmgo = config['helmgo']
        config.pop('helmgo')
        gao = GA(helmgo, **config)
        
        # Add initial data
        for seq, exp_score in zip(self._init_pop, self._init_scores):
            data.append((0, 0, exp_score, np.nan, seq.count('.'), seq))

        for i in range(n_sample):
            print('Run: %s' % (i + 1))

            # We want to keep a copy of the random peptides generated
            candidates = self._init_pop.copy()
            candidate_scores = self._init_scores.copy()

            # Compute the MAP4 fingerprint for all the peptides
            X_fps_exp = map4_fingerprint(candidates, input_type='helm')
            y_exp = torch.from_numpy(candidate_scores).float()
            print('Exp dataset size: (%d, %d)' % (X_fps_exp.shape[0], X_fps_exp.shape[1]))

            print('\n')
            print('Init.')
            print('N pep: ', X_fps_exp.shape[0])
            print('Best peptide: %.3f' % y_exp.min())
            for n in [-13, -12, -11, -10, -9, -8]:
                print('N pep under %d kcal/mol: %03d' % (n, y_exp[y_exp < n].shape[0]))
            print('Non binding pep        : %03d' % (y_exp[y_exp == 0.].shape[0]))
            print('\n')

            for j in range(n_step):
                print('Generation: %d' % (j + 1))
                
                # Fit GP model
                model = get_fitted_model(X_fps_exp, y_exp, kernel=TanimotoSimilarityKernel)

                # Initialize acquisition function
                #acq = AcqScoring(model, probability_of_improvement, y_exp, sequence_type='helm', greater_is_better=True)
                scoring_function = AcqScoring(model, expected_improvement, y_exp, sequence_type='helm', greater_is_better=False)

                # Find new candidates using GA optimization
                gao.run(scoring_function, candidates, candidate_scores)

                # Take NCAN (96) best candidates found
                candidate_sequences = gao.sequences[:NCAN]
                candidates_acq = gao.scores[:NCAN]

                clusters = defaultdict(list)
                for i_seq, sequence in enumerate(candidate_sequences):
                    clusters[sequence.count('.')].append(i_seq)
                print('Final selection:', ['%d: %d' % (k, len(v)) for k, v in clusters.items()])

                # Get affinitiy binding values (MAKE TEST)
                candidate_sequences_fasta = [''.join(c.split('$')[0].split('{')[1].split('}')[0].split('.')) for c in candidate_sequences]
                y_candidates = mps.predict_energy(candidate_sequences_fasta)

                # Add candidates to the training set
                candidates = np.append(candidates, candidate_sequences)
                candidate_scores = np.append(candidate_scores, y_candidates)
                candidate_fps = map4_fingerprint(candidate_sequences, input_type='helm')

                X_fps_exp = torch.cat([X_fps_exp, candidate_fps])
                y_exp = torch.cat([y_exp, torch.from_numpy(y_candidates)])
                
                print('')
                print('N pep: ', X_fps_exp.shape[0])
                print('Best peptide: %.3f' % y_exp.min())
                for n in [-13, -12, -11, -10, -9, -8]:
                    print('N pep under %d kcal/mol: %03d' % (n, y_exp[y_exp < n].shape[0]))
                print('Non binding pep        : %03d' % (y_exp[y_exp == 0.].shape[0]))
                print('')
                
                # Store data
                for seq, acq_score, exp_score in zip(candidate_sequences, candidates_acq, y_candidates):
                    data.append((i + 1, j + 1, exp_score, acq_score, seq.count('.'), seq))
        
        columns = ['sample', 'gen', 'exp_score', 'acq_score', 'length', 'sequence']
        df = pd.DataFrame(data=data, columns=columns)
        
        return df        

In [10]:
parameters = {'NCAN': 96,
              'n_gen': 1, 'helmgo': helmgo,
              'sequence_n_gen': 10, 'sequence_n_children': 500, 'sequence_sigma': 0.1, 'sequence_elitism': True, 'sequence_n_process': 4,
              'scaffold_n_gen': 1, 'scaffold_n_children': 1000, 'scaffold_sigma': 0.5, 'scaffold_elitism': True,
              'scaffold_probability': 0.25, 'scaffold_only_terminus': True, 'scaffold_minimum_size': 8, 'scaffold_maximum_size': 11}

dmt = DMTExperience(random_peptides, random_peptide_scores)
df = dmt.run(4, 10, **parameters)

Run: 1
Exp dataset size: (150, 4096)


Init.
N pep:  150
Best peptide: -8.158
N pep under -13 kcal/mol: 000
N pep under -12 kcal/mol: 000
N pep under -11 kcal/mol: 000
N pep under -10 kcal/mol: 000
N pep under -9 kcal/mol: 000
N pep under -8 kcal/mol: 002
Non binding pep        : 000


Generation: 1
End scaffold opt - Score: -0.011 - Seq: 9 - PEPTIDE1{F.W.Q.I.F.N.L.N.M}$$$$V2.0
End sequence opt - Score: -0.014 - Seq: 9 - PEPTIDE1{F.W.M.C.F.N.L.N.M}$$$$V2.0
End sequence opt - Score: -0.011 - Seq: 8 - PEPTIDE1{W.F.C.F.N.M.C.I}$$$$V2.0
End sequence opt - Score: -0.015 - Seq: 10 - PEPTIDE1{W.F.L.C.N.M.V.I.L.V}$$$$V2.0
End sequence opt - Score: -0.018 - Seq: 11 - PEPTIDE1{F.F.F.L.M.F.I.I.L.L.V}$$$$V2.0
End GA opt - Score: -0.018 - Seq: 11 - PEPTIDE1{F.F.F.L.M.F.I.I.L.L.V}$$$$V2.0
57.73698592185974
Final selection: ['11: 94', '10: 2']

N pep:  246
Best peptide: -9.647
N pep under -13 kcal/mol: 000
N pep under -12 kcal/mol: 000
N pep under -11 kcal/mol: 000
N pep under -10 kcal/mol: 000
N pep

End scaffold opt - Score: -0.014 - Seq: 10 - PEPTIDE1{F.L.I.M.F.V.I.F.G.V}$$$$V2.0
End sequence opt - Score: -0.000 - Seq: 8 - PEPTIDE1{M.M.L.F.I.Y.W.V}$$$$V2.0
End sequence opt - Score: -0.001 - Seq: 9 - PEPTIDE1{L.M.L.F.F.I.F.W.V}$$$$V2.0
End sequence opt - Score: -0.047 - Seq: 10 - PEPTIDE1{F.L.M.M.F.V.I.F.F.V}$$$$V2.0
End sequence opt - Score: -0.000 - Seq: 11 - PEPTIDE1{F.L.M.M.F.V.I.F.W.V.T}$$$$V2.0
End GA opt - Score: -0.047 - Seq: 10 - PEPTIDE1{F.L.M.M.F.V.I.F.F.V}$$$$V2.0
49.845787048339844
Final selection: ['10: 96']

N pep:  438
Best peptide: -13.169
N pep under -13 kcal/mol: 001
N pep under -12 kcal/mol: 075
N pep under -11 kcal/mol: 190
N pep under -10 kcal/mol: 224
N pep under -9 kcal/mol: 229
N pep under -8 kcal/mol: 288
Non binding pep        : 000

Generation: 4
End scaffold opt - Score: -0.006 - Seq: 10 - PEPTIDE1{F.L.M.M.F.L.L.F.F.V}$$$$V2.0


Process ForkPoolWorker-46:
Traceback (most recent call last):
  File "/home/eberhardt/Applications/miniconda3/envs/python3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/eberhardt/Applications/miniconda3/envs/python3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/eberhardt/Applications/miniconda3/envs/python3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/eberhardt/Applications/miniconda3/envs/python3/lib/python3.7/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
Process ForkPoolWorker-47:


KeyboardInterrupt: 

  File "/home/eberhardt/Documents/mobius/data/helm_ga/ga.py", line 156, in run
    self.sequences, self.scores = super().run(scoring_function, sequences, scores)
  File "/home/eberhardt/Documents/mobius/data/helm_ga/ga.py", line 108, in run
    scores = self._scoring_function.evaluate(sequences)
  File "/home/eberhardt/Documents/mobius/data/helm_ga/baye.py", line 126, in evaluate
    fps = map4_fingerprint(sequences, self._sequence_type)
Process ForkPoolWorker-45:
  File "/home/eberhardt/Documents/mobius/data/helm_ga/baye.py", line 109, in map4_fingerprint
    fps = MAP4_unf.calculate_many([f[input_type](s) for s in molecule_strings])
Traceback (most recent call last):
  File "/home/eberhardt/Applications/miniconda3/envs/python3/lib/python3.7/site-packages/map4/map4.py", line 64, in calculate_many
    return [self._fold(pairs) for pairs in atom_env_pairs_list]
  File "/home/eberhardt/Applications/miniconda3/envs/python3/lib/python3.7/site-packages/map4/map4.py", line 64, in <listcomp>
